In [1]:
import pandas as pd
import numpy as np

In [42]:
df = pd.read_csv('DMV Registration.csv')

# Remove rows that is not california county
df = df[df['County']!='Out Of State']

# Year from 2010 to 2020 only
df = df[df['Data Year']>=2010]

# Keep only Electrical Vehicles in the dataset
#df = df[df['Fuel Type']=='Electric']

df

,Data Year,County,Fuel Type,MAKE,MODEL,Number of Vehicles
41,2010,Alameda,Electric,FORD,RANGER,1
42,2010,Alameda,Electric,TESLA,ROADSTER,12
43,2010,Amador,Electric,FORD,RANGER,1
44,2010,Contra Costa,Electric,TESLA,ROADSTER,9
45,2010,Fresno,Electric,TESLA,ROADSTER,2
...,...,...,...,...,...,...
10029,2020,Yuba,Electric,TESLA,Model X,1
10030,2020,Yuba,Electric,TESLA,Model Y,9
10031,2020,Yuba,PHEV,Chrysler,Pacifica Hybrid,3
10032,2020,Yuba,PHEV,Hyundai,Ioniq Plug-in Hybrid,1


In [43]:
set(df['Fuel Type'])

{'Electric', 'Hydrogen', 'PHEV'}

In [53]:
# Original Dataset number of vehicles registered of county and year 
df_new = df.copy()
df_new = df_new[df_new['Fuel Type']=='Electric']
cd = df_new.groupby(['Data Year', 'County'])['Number of Vehicles'].agg(['sum']).reset_index()
cd

,Data Year,County,sum
0,2010,Alameda,13
1,2010,Amador,1
2,2010,Contra Costa,9
3,2010,Fresno,2
4,2010,Humboldt,1
...,...,...,...
539,2020,Tulare,195
540,2020,Tuolumne,38
541,2020,Ventura,2245
542,2020,Yolo,375


In [51]:
# List of all the counties
c = pd.Series(list(set(df['County']))).to_frame()
c = c.rename(columns={0:'county'})

# List of all the years
d = pd.Series(list(range(2010,2021))).to_frame()
d = d.rename(columns={0:'year'})

# All combinations of Couny and Year
cp = c.assign(key=0).merge(d.assign(key=0), how='left', on = 'key')
cp = cp.drop(['key'], axis=1)
cp

,county,year
0,Fresno,2010
1,Fresno,2011
2,Fresno,2012
3,Fresno,2013
4,Fresno,2014
...,...,...
633,Lassen,2016
634,Lassen,2017
635,Lassen,2018
636,Lassen,2019


In [58]:
# Left join the datatables together to create a raw final dataset
df_final = cp.merge(cd, left_on=['county','year'], right_on=['County','Data Year'], how='left')
df_final = df_final.drop(['Data Year','County'], axis=1)

# Fill NaN values to 0
df_final['sum'] = df_final['sum'].fillna(0) 

# Change data type of value in sum column into int
df_final['sum'] = df_final['sum'].astype(int)

# Rename the dependent variable column
df_final = df_final.rename(columns={'sum':'num_ev'})

df_final

,county,year,num_ev
0,Fresno,2010,2
1,Fresno,2011,17
2,Fresno,2012,20
3,Fresno,2013,85
4,Fresno,2014,444
...,...,...,...
633,Lassen,2016,0
634,Lassen,2017,1
635,Lassen,2018,1
636,Lassen,2019,1


In [60]:
# Save as a new .csv file
df_final.to_csv ('numEV_CountyandYear.csv', index = True, header=True)